In [7]:

#EXO1

from datetime import datetime, timedelta
import calendar

def generer_echeancier_simple(date_debut, frequence_mois, duree_annees, convention):
    """Génère un échéancier des paiements avec une version simplifiée."""
    echeancier = []
    date_actuelle = date_debut

    for i in range(1, duree_annees * 12 // frequence_mois + 1): # + 1 est ajouté pour inclure le dernier paiement, car la fonction range exclut la valeur finale.
        # Calcul de la date de fin (ajout de mois)
        date_fin = ajouter_mois(date_actuelle, frequence_mois)
        
        # Ajustement selon Modified Following
        date_fin_ajustee = ajuster_modified_following(date_fin)
        
        # Calcul de la fraction d'année selon la convention
        fraction_annee = calculer_fraction_annee_simple(date_actuelle, date_fin_ajustee, convention)
        
        # Ajout à l'échéancier
        echeancier.append({
            "N° Coupon": i,
            "Date Début": date_actuelle.strftime("%d/%m/%Y"),
            "Date Fin": date_fin_ajustee.strftime("%d/%m/%Y"),
            "Fraction Année": round(fraction_annee, 6)
        })
        
        # Mise à jour de la date de début pour la prochaine période
        date_actuelle = date_fin_ajustee

    return echeancier

def ajouter_mois(date_source, mois):
    """Ajoute un certain nombre de mois à une date."""
    mois_total = date_source.month - 1 + mois
    annee = date_source.year + mois_total // 12
    mois = mois_total % 12 + 1
    jour = min(date_source.day, calendar.monthrange(annee, mois)[1])
    return datetime(annee, mois, jour)

def ajuster_modified_following(date):
    """Ajuste la date pour qu'elle soit un jour ouvré, selon la règle Modified Following."""
    if date.weekday() in (5, 6):  # Si samedi (5) ou dimanche (6)
        date += timedelta(days=(7 - date.weekday()))  # Déplace au lundi
    return date

def calculer_fraction_annee_simple(date_debut, date_fin, convention):
    """Calcule la fraction d'année selon une convention simplifiée."""
    jours_ecoules = (date_fin - date_debut).days
    if convention == "ACT/365":
        return jours_ecoules / 365
    elif convention == "ACT/360":
        return jours_ecoules / 360
    elif convention == "30/360":
        return jours_ecoules / 360
    else:
        raise ValueError("Convention de décompte de jours inconnue.")

# Exemple d'utilisation
date_debut = datetime(2024, 1, 5)  # Date de début
frequence_mois = 3                 # Fréquence trimestrielle
duree_annees = 5                   # Durée de 5 ans

# Générer l'échéancier pour chaque convention et afficher
for convention in ["ACT/365", "ACT/360", "30/360"]:
    print(f"\nÉchéancier pour la convention {convention} :")
    echeancier = generer_echeancier_simple(date_debut, frequence_mois, duree_annees, convention)
    for ligne in echeancier:
        print(ligne)


Échéancier pour la convention ACT/365 :
{'N° Coupon': 1, 'Date Début': '05/01/2024', 'Date Fin': '05/04/2024', 'Fraction Année': 0.249315}
{'N° Coupon': 2, 'Date Début': '05/04/2024', 'Date Fin': '05/07/2024', 'Fraction Année': 0.249315}
{'N° Coupon': 3, 'Date Début': '05/07/2024', 'Date Fin': '07/10/2024', 'Fraction Année': 0.257534}
{'N° Coupon': 4, 'Date Début': '07/10/2024', 'Date Fin': '07/01/2025', 'Fraction Année': 0.252055}
{'N° Coupon': 5, 'Date Début': '07/01/2025', 'Date Fin': '07/04/2025', 'Fraction Année': 0.246575}
{'N° Coupon': 6, 'Date Début': '07/04/2025', 'Date Fin': '07/07/2025', 'Fraction Année': 0.249315}
{'N° Coupon': 7, 'Date Début': '07/07/2025', 'Date Fin': '07/10/2025', 'Fraction Année': 0.252055}
{'N° Coupon': 8, 'Date Début': '07/10/2025', 'Date Fin': '07/01/2026', 'Fraction Année': 0.252055}
{'N° Coupon': 9, 'Date Début': '07/01/2026', 'Date Fin': '07/04/2026', 'Fraction Année': 0.246575}
{'N° Coupon': 10, 'Date Début': '07/04/2026', 'Date Fin': '07/07/202

In [8]:
#EXO2
from datetime import datetime, timedelta

# Liste des jours fériés (exemple simple)
jours_feries = ["01/01", "04/07", "25/12"]

# Ajustement suivant la méthode 'Following'
def ajuster_following(date):
    """ Ajuste la date pour qu'elle tombe le jour ouvré suivant """
    while date.weekday() == 5:  # Samedi
        date += timedelta(days=2)  # Déplacer au lundi
    while date.weekday() == 6:  # Dimanche
        date += timedelta(days=1)  # Déplacer au lundi
    return date

# Fonction pour ajouter des mois à une date
def ajouter_mois(date, mois):
    """ Ajoute un certain nombre de mois à la date """
    annee = date.year + (date.month + mois - 1) // 12
    mois_final = (date.month + mois - 1) % 12 + 1
    jour = min(date.day, [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31][mois_final-1])
    return datetime(annee, mois_final, jour)

# Calcul de la fraction d'année selon la convention 30/360
def calculer_fraction_annee(date_debut, date_fin):
    """ Calcule la fraction d'année selon la convention 30/360 """
    return (date_fin.month - date_debut.month) * 30 / 360

# Générer l'échéancier
def generer_echeancier(date_debut, duree_annees, ajustement="Following"):
    echeancier = []
    date_actuelle = date_debut

    for i in range(1, duree_annees * 12 + 1):
        # Ajouter 1 mois à la date actuelle
        date_fin = ajouter_mois(date_actuelle, 1)
        
        # Appliquer l'ajustement des jours fériés
        if ajustement == "Following":
            date_fin = ajuster_following(date_fin)

        # Calcul de la fraction d'année
        fraction_annee = calculer_fraction_annee(date_actuelle, date_fin)

        # Ajouter au tableau de l'échéancier
        echeancier.append({
            "N° Coupon": i,
            "Date Début": date_actuelle.strftime("%d/%m/%Y"),
            "Date Fin": date_fin.strftime("%d/%m/%Y"),
            "Fraction Année": round(fraction_annee, 6)
        })
        
        # Mettre à jour la date de début pour le coupon suivant
        date_actuelle = date_fin

    return echeancier

# Exemple d'utilisation
date_debut = datetime(2024, 1, 31)
duree_annees = 2  # Durée de 2 ans

# Générer l'échéancier pour la méthode "Following"
echeancier = generer_echeancier(date_debut, duree_annees, ajustement="Following")

# Affichage du tableau d'échéancier
for ligne in echeancier:
    print(ligne)

{'N° Coupon': 1, 'Date Début': '31/01/2024', 'Date Fin': '28/02/2024', 'Fraction Année': 0.083333}
{'N° Coupon': 2, 'Date Début': '28/02/2024', 'Date Fin': '28/03/2024', 'Fraction Année': 0.083333}
{'N° Coupon': 3, 'Date Début': '28/03/2024', 'Date Fin': '29/04/2024', 'Fraction Année': 0.083333}
{'N° Coupon': 4, 'Date Début': '29/04/2024', 'Date Fin': '29/05/2024', 'Fraction Année': 0.083333}
{'N° Coupon': 5, 'Date Début': '29/05/2024', 'Date Fin': '01/07/2024', 'Fraction Année': 0.166667}
{'N° Coupon': 6, 'Date Début': '01/07/2024', 'Date Fin': '01/08/2024', 'Fraction Année': 0.083333}
{'N° Coupon': 7, 'Date Début': '01/08/2024', 'Date Fin': '02/09/2024', 'Fraction Année': 0.083333}
{'N° Coupon': 8, 'Date Début': '02/09/2024', 'Date Fin': '02/10/2024', 'Fraction Année': 0.083333}
{'N° Coupon': 9, 'Date Début': '02/10/2024', 'Date Fin': '04/11/2024', 'Fraction Année': 0.083333}
{'N° Coupon': 10, 'Date Début': '04/11/2024', 'Date Fin': '04/12/2024', 'Fraction Année': 0.083333}
{'N° Coup

In [9]:
print (date_source)

NameError: name 'date_source' is not defined